<h1> Tashkeel </h1>

This project is developed by Abdulhameed Osama, Hossam Nabil and Nourhan Mohamed as a part of the Natural Language Processing course (NLP) at Cairo University.

In this project a Bi-LSTM model is trained to predict the diacritics of Arabic text. The model is trained on a dataset of 18 million characters from various domains. The model is trained on Google Colab using a Tesla K80 GPU.

We achieved an accuracy of 96.5% on the test set.

The model starts with a character embedding layer, followed by a Bi-LSTM layer, then a dense layer and finally a softmax layer. The model is trained using the Adam optimizer and the categorical crossentropy loss function.

<h3> Imports </h3>

In [65]:
import numpy as np
import pickle as pkl
import helper_file as hf

# Arabic Text Preprocessing
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")

from nltk.tokenize import word_tokenize, sent_tokenize

# import pyarabic.araby as araby
# import pyarabic.number as number
# from pyarabic.araby import strip_tashkeel, strip_tatweel, normalize_ligature
# from pyarabic.araby import tokenize, is_arabicrange, is_arabicword

# Arabic Stopwords
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer


# Arabic Text Preprocessing

In [66]:
# Read the dataset
dataset = hf.read_file('dataset/train.txt')

arabic_letters = hf.read_pickle('Delivery/arabic_letters.pickle')

diacritics = hf.read_pickle('Delivery/diacritics.pickle')

diacritics_to_id = hf.read_pickle('Delivery/diacritics2id.pickle')

In [67]:
# Preprocess the dataset using bert tokenizer
# tokenized_text = tokenizer.tokenize(dataset)
# hf.write_pickle('tokenized_text.pickle', tokenized_text)

tokenized_sentences = sent_tokenize(dataset)
hf.write_pickle('tokenized_text_nltk.pickle', tokenized_sentences)

In [58]:
bert = hf.read_pickle('tokenized_text.pickle')  # bert tokenized text
nltk = hf.read_pickle('tokenized_text_nltk.pickle')  # nltk tokenized text

print(bert[:1000])
print(nltk[:1000])

['قوله', ':', '(', 'او', 'قطع', 'الاول', 'يده', 'الخ', ')', 'قال', 'الزر', '##كش', '##ي', '(', '14', '/', '123', ')', 'ابن', 'عرفة', ':', 'قوله', ':', 'بلفظ', 'يقتضي', '##ه', 'كان', '##كار', 'غير', 'حديث', 'بالاسلام', 'وجوب', 'ما', 'علم', 'وجوب', '##ه', 'من', 'الدين', 'ضرورة', '(', 'كال', '##قاء', 'مصحف', 'بق', '##ذر', 'وشد', 'زن', '##ار', ')', 'ابن', 'عرفة', ':', 'قول', 'ابن', 'شاس', ':', 'او', 'بفعل', 'يتضمن', '##ه', 'هو', 'كلب', '##س', 'الزنا', '##ر', 'والق', '##اء', 'المصحف', 'في', 'صريح', 'النج', '##اسة', 'والس', '##جود', 'للص', '##نم', 'ونحو', 'ذلك', '(', 'وس', '##حر', ')', 'محمد', ':', 'قول', 'مالك', 'واصحابه', 'ان', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', ':', 'هو', 'كال', '##زن', '##ديق', 'اذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يست', '##تب', '.', '(', 'قوله', 'لعدم', 'ما', 'تتعلق', 'الخ', ')', 'اي', 'الوصية', '(', 'قوله', 'ما', 'مر', ')', 'اي', 'قبيل', 'قول', 'المتن', 'لغ', '##ت', 'ولو', 'اقتص', '##ر', 'على', 'اوصي', '##ت', 'له', 'بش', '##اة', 'او', 'اعط', '#

In [83]:
import re

# Clean the original dataset
cleaned_dataset = hf.clean_dataset(dataset[:5000])
print(cleaned_dataset)

قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ أَيْ الْوَصِيَّةُ قَوْلُهُ مَا مَرَّ أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِب